In [1]:
!pip install monai
!pip install -U scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import sys
import os

py_file_location = "/content/drive/MyDrive/liver-imaging-analysis/engine"
sys.path.append(os.path.abspath(py_file_location))

import dataloader 
from monai.transforms import(LoadImageD,Compose,FlipD)

import torch
import nibabel as nib
import numpy as np

import json
#Read configurations from JSON File
with open('/content/drive/MyDrive/liver-imaging-analysis/config/configs.json') as f:
    config = json.load(f)


In [4]:
################### loading dataset#######################
trans=Compose([LoadImageD(("image", "label")),FlipD(("image", "label"), spatial_axis=[-2,-3])])

loader=dataloader.DataLoader(dataset_path ='/content/drive/MyDrive/15lits',
                             batch_size=1,transforms=trans ,num_workers=0,
                             pin_memory=False,test_size=0)
                             
dataset=loader.get_training_data()
###############pathes for saving###########################
volume_save_path = '/content/drive/MyDrive/nehaha/volume/'
mask_save_path   = '/content/drive/MyDrive/nehaha/mask/'

In [5]:
def xy_volume_indicies(loaded_volumes):
  '''
  get the x,y minimum and x,y maximum that contain the mask 
  loaded_volumes: is the dataset which comes from the dataloader
  it returns xminimum,xmaximum,yminimum,ymaximum for the mask
  '''
  xminimum=0
  xmaximum=0
  yminimum=0
  ymaximum=0

  shape_x=0
  shape_y=0
  for data in loaded_volumes:
    shape_x=data['label'][0].shape[0]
    shape_y=data['label'][0].shape[1]
    break

  sum_of_zyplanes=torch.zeros(shape_x,shape_y)
  for data in loaded_volumes:
    sum_slices = torch.sum(data['label'][0], dim = 2, keepdim=False)
    # print(sum_slices.shape)  
    sum_of_zyplanes += sum_slices
  # print(sum_of_zyplanes.shape) 
  xrange = torch.sum(sum_of_zyplanes, dim = 1, keepdim=False)
  yrange=  torch.sum(sum_of_zyplanes, dim = 0, keepdim=False)
  # print("x range",xrange)
  # print('y range',yrange) 

  index=0
  for value in xrange:
    if value != 0:
      xminimum=index
      break
    index +=1

  index=0
  for value in yrange:
    if value !=0:
      yminimum=index
      break
    index +=1

  index= xrange.shape[0]-1
  for value in reversed(xrange):
    if value !=0:
      xmaximum= index
      break
    index -=1

  index= yrange.shape[0]-1
  for value in reversed(yrange):
    if value !=0:
      ymaximum=index
      break  
    index -=1

  return xminimum,xmaximum,yminimum,ymaximum;

In [6]:
def get_margin(pad_Percentage,size_of_plane):
  '''
  get the padding that i want in axis as 15pixel before the organ and 15pixel after the organ in for example x direction 
  pad_Percentage: the padding that we want to keep 
  size_of_plane: like if it is x axis it will be like 512
  '''
  margin=int(pad_Percentage*(size_of_plane)/100) 
  if margin%2 == 1:
    margin=margin+1
  margin=margin/2 
  return margin 

In [7]:
def get_cropping_indices(margin,min,max,size_plane,symetric_cropping_flag=1):
  '''
  get the idnicies for the cropping and 
  margin: the numper of pixels that we will pad with it on each axis
  min: min index for the mask in thap axis 
  max: max index for the mask in thap axis 
  size_plane: like if it is x axis it will be like 512
  symetric_cropping_flag: must be 1 in x and y axises and for z(depth) in could be 0 to cut in one direction but if it 0 the cutting will be symetric
  '''
  min_crop_index=0
  max_crop_index=0

  if (min-margin)<0: 

    if(min-(size_plane-1-max)):###### doesn't make sense cuz it means that u want to keep a padding bigger than the image
      min_crop_index=0
      max_crop_index=size_plane-1     
      return min_crop_index , max_crop_index;

    else:
      min_crop_index=0
    
  else:
    min_crop_index=(min-margin)

  if (max+margin)>size_plane-1:

    if(min-(size_plane-1-max)):

      if(symetric_cropping_flag):
        max_crop_index=size_plane-1
        min_crop_index=min-(size_plane-1-max)
        return min_crop_index , max_crop_index;

      else: ###########make a cropping from one side only
        max_crop_index=size_plane-1
        return min_crop_index , max_crop_index;

    else:###### doesn't make sense cuz it means that u want to keep a padding bigger than the image
      max_crop_index=size_plane-1 
      return min_crop_index , max_crop_index;
  else:
    max_crop_index=max+margin
    return min_crop_index , max_crop_index;

In [8]:
def cropping(volume_save_path,mask_save_path,dataset,symetric_crop,xpadding_percentage,ypadding_percentage,zpadding_percentage):
  '''
  crop the volume and print some information like this:
    original shape of data torch.Size([456, 449, 147])
    mask XMin 86 liver XMax 431 mask YMin 104 mask YMax 377 mask ZMin 61 mask ZMax 139
    xpad 2.0 ypad 2.0 zpad 4.0
    x cropping range 84.0 to 433.0 y cropping range 102.0 to 379.0 z cropping range 57.0 to 143.0
    cropped_mask torch.Size([349, 277, 86])
    cropped_volume torch.Size([349, 277, 86])

  volume_save_path:volume saving path
  mask_save_path:mask saving path
  dataset: which loaded
  symetric_crop: 1 for a symetric cropping in depth and 0 for a normal cropping
  xpadding_percentage: the percentage of padding that we need
  ypadding_percentage: the percentage of padding that we need
  zpadding_percentage: the percentage of padding that we need
  '''

  xminimum,xmaximum,yminimum,ymaximum=xy_volume_indicies(dataset)

  data_naming_index=0
  for data in dataset:
    zminimum=0
    zmaximum=0
    print("#############",data_naming_index,"#############")
    print("original shape of data",data['label'][0].shape)
    xzplane=torch.sum(data['label'][0],dim=1,keepdim=False)
    zplane=torch.sum(xzplane,dim=0,keepdim=False)

    index=0
    for value in zplane:
      if value != 0:
        zminimum =index
        break
      index +=1

    index= zplane.shape[0] -1    
    for value in reversed(zplane):
      if value != 0:
        zmaximum =index
        break       
      index -= 1

    x_margin=get_margin(xpadding_percentage,data['label'][0].shape[0])
    y_margin=get_margin(ypadding_percentage,data['label'][0].shape[1])
    z_margin=get_margin(zpadding_percentage,data['label'][0].shape[2])

    xmin,xmax =get_cropping_indices(x_margin,xminimum,xmaximum,data['label'][0].shape[0],1)
    ymin,ymax =get_cropping_indices(y_margin,yminimum,ymaximum,data['label'][0].shape[1],1)
    zmin,zmax =get_cropping_indices(z_margin,zminimum,zmaximum,data['label'][0].shape[2],symetric_crop) #####0 cuz i don't wanna a symetric croppin in z

    print('liver XMin',xminimum,'liver XMax',xmaximum,'liver YMin', yminimum,'liver YMax',ymaximum,'liver ZMin', zminimum,'liver ZMax',zmaximum ) 
    print("xpad",x_margin,"ypad",y_margin,"zpad",z_margin)
    print("x cropping range",xmin,"to",xmax,"y cropping range",ymin,"to",ymax,"z cropping range",zmin,"to",zmax)

    cropped_mask=data['label'][0][int(xmin):int(xmax) , int(ymin):int(ymax), int(zmin):int(zmax)]
    cropped_volume=data['image'][0][int(xmin):int(xmax) , int(ymin):int(ymax), int(zmin):int(zmax)]
    print("cropped_mask",cropped_mask.shape)
    print("cropped_volume",cropped_volume.shape)

    nii_crop_vloume = nib.Nifti1Image(cropped_volume.numpy(), affine=np.eye(4))
    nii_crop_mask = nib.Nifti1Image(cropped_mask.numpy(), affine=np.eye(4))
    nii_volume_path = os.path.join(volume_save_path, 'volume-'  + str(data_naming_index))+'.nii'
    nii_mask_path   = os.path.join(mask_save_path, 'segmentation-' + str(data_naming_index))+'.nii'
    nib.save(nii_crop_vloume , nii_volume_path) 
    nib.save(nii_crop_mask   , nii_mask_path) 

    # if data_naming_index ==0:  ##################
    #   break                    ##################
    data_naming_index += 1



In [9]:
cropping(volume_save_path,mask_save_path,dataset,0,1,1,5)

############# 0 #############
original shape of data torch.Size([512, 512, 244])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 157 liver ZMax 235
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 151.0 to 241.0
cropped_mask torch.Size([435, 326, 90])
cropped_volume torch.Size([435, 326, 90])
############# 1 #############
original shape of data torch.Size([512, 512, 193])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 125 liver ZMax 185
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 120.0 to 190.0
cropped_mask torch.Size([435, 326, 70])
cropped_volume torch.Size([435, 326, 70])
############# 2 #############
original shape of data torch.Size([512, 512, 188])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 89 liver ZMax 177
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0

**training set for lits**
############# 0 #############
original shape of data torch.Size([512, 512, 75])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 45 liver ZMax 73
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 43.0 to 74
cropped_mask torch.Size([465, 406, 31])
cropped_volume torch.Size([465, 406, 31])
############# 1 #############
original shape of data torch.Size([512, 512, 123])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 45 liver ZMax 73
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 42.0 to 76.0
cropped_mask torch.Size([465, 406, 34])
cropped_volume torch.Size([465, 406, 34])
############# 2 #############
original shape of data torch.Size([512, 512, 501])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 295 liver ZMax 475
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 282.0 to 488.0
cropped_mask torch.Size([465, 406, 206])
cropped_volume torch.Size([465, 406, 206])
############# 3 #############
original shape of data torch.Size([512, 512, 685])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 387 liver ZMax 662
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 370.0 to 679.0
cropped_mask torch.Size([465, 406, 309])
cropped_volume torch.Size([465, 406, 309])
############# 4 #############
original shape of data torch.Size([512, 512, 683])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 387 liver ZMax 645
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 370.0 to 662.0
cropped_mask torch.Size([465, 406, 292])
cropped_volume torch.Size([465, 406, 292])
############# 5 #############
original shape of data torch.Size([512, 512, 677])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 322 liver ZMax 634
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 305.0 to 651.0
cropped_mask torch.Size([465, 406, 346])
cropped_volume torch.Size([465, 406, 346])
############# 6 #############
original shape of data torch.Size([512, 512, 683])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 422 liver ZMax 635
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 405.0 to 652.0
cropped_mask torch.Size([465, 406, 247])
cropped_volume torch.Size([465, 406, 247])
############# 7 #############
original shape of data torch.Size([512, 512, 781])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 273 liver ZMax 466
xpad 3.0 ypad 3.0 zpad 20.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 253.0 to 486.0
cropped_mask torch.Size([465, 406, 233])
cropped_volume torch.Size([465, 406, 233])
############# 8 #############
original shape of data torch.Size([512, 512, 986])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 387 liver ZMax 625
xpad 3.0 ypad 3.0 zpad 25.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 362.0 to 650.0
cropped_mask torch.Size([465, 406, 288])
cropped_volume torch.Size([465, 406, 288])
############# 9 #############
original shape of data torch.Size([512, 512, 771])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 326 liver ZMax 493
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 307.0 to 512.0
cropped_mask torch.Size([465, 406, 205])
cropped_volume torch.Size([465, 406, 205])
############# 10 #############
original shape of data torch.Size([512, 512, 771])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 266 liver ZMax 513
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 247.0 to 532.0
cropped_mask torch.Size([465, 406, 285])
cropped_volume torch.Size([465, 406, 285])
############# 11 #############
original shape of data torch.Size([512, 512, 856])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 412 liver ZMax 616
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 391.0 to 637.0
cropped_mask torch.Size([465, 406, 246])
cropped_volume torch.Size([465, 406, 246])
############# 12 #############
original shape of data torch.Size([512, 512, 756])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 288 liver ZMax 481
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 269.0 to 500.0
cropped_mask torch.Size([465, 406, 231])
cropped_volume torch.Size([465, 406, 231])
############# 13 #############
original shape of data torch.Size([512, 512, 466])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 272 liver ZMax 438
xpad 3.0 ypad 3.0 zpad 12.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 260.0 to 450.0
cropped_mask torch.Size([465, 406, 190])
cropped_volume torch.Size([465, 406, 190])
############# 14 #############
original shape of data torch.Size([512, 512, 816])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 344 liver ZMax 532
xpad 3.0 ypad 3.0 zpad 20.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 324.0 to 552.0
cropped_mask torch.Size([465, 406, 228])
cropped_volume torch.Size([465, 406, 228])
############# 15 #############
original shape of data torch.Size([512, 512, 761])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 231 liver ZMax 463
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 212.0 to 482.0
cropped_mask torch.Size([465, 406, 270])
cropped_volume torch.Size([465, 406, 270])
############# 16 #############
original shape of data torch.Size([512, 512, 751])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 291 liver ZMax 481
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 272.0 to 500.0
cropped_mask torch.Size([465, 406, 228])
cropped_volume torch.Size([465, 406, 228])
############# 17 #############
original shape of data torch.Size([512, 512, 836])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 409 liver ZMax 578
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 388.0 to 599.0
cropped_mask torch.Size([465, 406, 211])
cropped_volume torch.Size([465, 406, 211])
############# 18 #############
original shape of data torch.Size([512, 512, 846])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 364 liver ZMax 578
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 343.0 to 599.0
cropped_mask torch.Size([465, 406, 256])
cropped_volume torch.Size([465, 406, 256])
############# 19 #############
original shape of data torch.Size([512, 512, 846])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 411 liver ZMax 596
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 390.0 to 617.0
cropped_mask torch.Size([465, 406, 227])
cropped_volume torch.Size([465, 406, 227])
############# 20 #############
original shape of data torch.Size([512, 512, 908])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 447 liver ZMax 665
xpad 3.0 ypad 3.0 zpad 23.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 424.0 to 688.0
cropped_mask torch.Size([465, 406, 264])
cropped_volume torch.Size([465, 406, 264])
############# 21 #############
original shape of data torch.Size([512, 512, 836])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 278 liver ZMax 537
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 257.0 to 558.0
cropped_mask torch.Size([465, 406, 301])
cropped_volume torch.Size([465, 406, 301])
############# 22 #############
original shape of data torch.Size([512, 512, 427])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 196 liver ZMax 317
xpad 3.0 ypad 3.0 zpad 11.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 185.0 to 328.0
cropped_mask torch.Size([465, 406, 143])
cropped_volume torch.Size([465, 406, 143])
############# 23 #############
original shape of data torch.Size([512, 512, 461])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 201 liver ZMax 332
xpad 3.0 ypad 3.0 zpad 12.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 189.0 to 344.0
cropped_mask torch.Size([465, 406, 155])
cropped_volume torch.Size([465, 406, 155])
############# 24 #############
original shape of data torch.Size([512, 512, 455])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 265 liver ZMax 453
xpad 3.0 ypad 3.0 zpad 11.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 254.0 to 454
cropped_mask torch.Size([465, 406, 200])
cropped_volume torch.Size([465, 406, 200])
############# 25 #############
original shape of data torch.Size([512, 512, 424])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 153 liver ZMax 272
xpad 3.0 ypad 3.0 zpad 11.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 142.0 to 283.0
cropped_mask torch.Size([465, 406, 141])
cropped_volume torch.Size([465, 406, 141])
############# 26 #############
original shape of data torch.Size([512, 512, 463])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 204 liver ZMax 316
xpad 3.0 ypad 3.0 zpad 12.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 192.0 to 328.0
cropped_mask torch.Size([465, 406, 136])
cropped_volume torch.Size([465, 406, 136])
############# 27 #############
original shape of data torch.Size([512, 512, 422])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 156 liver ZMax 273
xpad 3.0 ypad 3.0 zpad 11.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 145.0 to 284.0
cropped_mask torch.Size([465, 406, 139])
cropped_volume torch.Size([465, 406, 139])
############# 28 #############
original shape of data torch.Size([512, 512, 432])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 185 liver ZMax 299
xpad 3.0 ypad 3.0 zpad 11.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 174.0 to 310.0
cropped_mask torch.Size([465, 406, 136])
cropped_volume torch.Size([465, 406, 136])
############# 29 #############
original shape of data torch.Size([512, 512, 407])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 165 liver ZMax 276
xpad 3.0 ypad 3.0 zpad 10.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 155.0 to 286.0
cropped_mask torch.Size([465, 406, 131])
cropped_volume torch.Size([465, 406, 131])
############# 30 #############
original shape of data torch.Size([512, 512, 410])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 163 liver ZMax 274
xpad 3.0 ypad 3.0 zpad 10.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 153.0 to 284.0
cropped_mask torch.Size([465, 406, 131])
cropped_volume torch.Size([465, 406, 131])
############# 31 #############
original shape of data torch.Size([512, 512, 401])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 176 liver ZMax 288
xpad 3.0 ypad 3.0 zpad 10.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 166.0 to 298.0
cropped_mask torch.Size([465, 406, 132])
cropped_volume torch.Size([465, 406, 132])
############# 32 #############
original shape of data torch.Size([512, 512, 987])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 477 liver ZMax 703
xpad 3.0 ypad 3.0 zpad 25.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 452.0 to 728.0
cropped_mask torch.Size([465, 406, 276])
cropped_volume torch.Size([465, 406, 276])
############# 33 #############
original shape of data torch.Size([512, 512, 654])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 300 liver ZMax 591
xpad 3.0 ypad 3.0 zpad 16.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 284.0 to 607.0
cropped_mask torch.Size([465, 406, 323])
cropped_volume torch.Size([465, 406, 323])
############# 34 #############
original shape of data torch.Size([512, 512, 338])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 37 liver ZMax 313
xpad 3.0 ypad 3.0 zpad 8.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 29.0 to 321.0
cropped_mask torch.Size([465, 406, 292])
cropped_volume torch.Size([465, 406, 292])
############# 35 #############
original shape of data torch.Size([512, 512, 605])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 272 liver ZMax 413
xpad 3.0 ypad 3.0 zpad 15.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 257.0 to 428.0
cropped_mask torch.Size([465, 406, 171])
cropped_volume torch.Size([465, 406, 171])
############# 36 #############
original shape of data torch.Size([512, 512, 624])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 336 liver ZMax 576
xpad 3.0 ypad 3.0 zpad 16.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 320.0 to 592.0
cropped_mask torch.Size([465, 406, 272])
cropped_volume torch.Size([465, 406, 272])
############# 37 #############
original shape of data torch.Size([512, 512, 588])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 260 liver ZMax 398
xpad 3.0 ypad 3.0 zpad 15.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 245.0 to 413.0
cropped_mask torch.Size([465, 406, 168])
cropped_volume torch.Size([465, 406, 168])
############# 38 #############
original shape of data torch.Size([512, 512, 565])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 240 liver ZMax 372
xpad 3.0 ypad 3.0 zpad 14.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 226.0 to 386.0
cropped_mask torch.Size([465, 406, 160])
cropped_volume torch.Size([465, 406, 160])
############# 39 #############
original shape of data torch.Size([512, 512, 689])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 267 liver ZMax 453
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 250.0 to 470.0
cropped_mask torch.Size([465, 406, 220])
cropped_volume torch.Size([465, 406, 220])
############# 40 #############
original shape of data torch.Size([512, 512, 826])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 328 liver ZMax 525
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 307.0 to 546.0
cropped_mask torch.Size([465, 406, 239])
cropped_volume torch.Size([465, 406, 239])
############# 41 #############
original shape of data torch.Size([512, 512, 845])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 354 liver ZMax 542
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 333.0 to 563.0
cropped_mask torch.Size([465, 406, 230])
cropped_volume torch.Size([465, 406, 230])
############# 42 #############
original shape of data torch.Size([512, 512, 547])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 324 liver ZMax 511
xpad 3.0 ypad 3.0 zpad 14.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 310.0 to 525.0
cropped_mask torch.Size([465, 406, 215])
cropped_volume torch.Size([465, 406, 215])
############# 43 #############
original shape of data torch.Size([512, 512, 517])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 334 liver ZMax 472
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 321.0 to 485.0
cropped_mask torch.Size([465, 406, 164])
cropped_volume torch.Size([465, 406, 164])
############# 44 #############
original shape of data torch.Size([512, 512, 574])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 354 liver ZMax 547
xpad 3.0 ypad 3.0 zpad 14.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 340.0 to 561.0
cropped_mask torch.Size([465, 406, 221])
cropped_volume torch.Size([465, 406, 221])
############# 45 #############
original shape of data torch.Size([512, 512, 437])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 260 liver ZMax 434
xpad 3.0 ypad 3.0 zpad 11.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 249.0 to 436
cropped_mask torch.Size([465, 406, 187])
cropped_volume torch.Size([465, 406, 187])
############# 46 #############
original shape of data torch.Size([512, 512, 247])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 103 liver ZMax 148
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 97.0 to 154.0
cropped_mask torch.Size([465, 406, 57])
cropped_volume torch.Size([465, 406, 57])
############# 47 #############
original shape of data torch.Size([512, 512, 391])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 262 liver ZMax 377
xpad 3.0 ypad 3.0 zpad 10.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 252.0 to 387.0
cropped_mask torch.Size([465, 406, 135])
cropped_volume torch.Size([465, 406, 135])
############# 48 #############
original shape of data torch.Size([512, 512, 276])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 141 liver ZMax 258
xpad 3.0 ypad 3.0 zpad 7.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 134.0 to 265.0
cropped_mask torch.Size([465, 406, 131])
cropped_volume torch.Size([465, 406, 131])
############# 49 #############
original shape of data torch.Size([512, 512, 601])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 341 liver ZMax 572
xpad 3.0 ypad 3.0 zpad 15.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 326.0 to 587.0
cropped_mask torch.Size([465, 406, 261])
cropped_volume torch.Size([465, 406, 261])
############# 50 #############
original shape of data torch.Size([512, 512, 668])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 255 liver ZMax 446
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 238.0 to 463.0
cropped_mask torch.Size([465, 406, 225])
cropped_volume torch.Size([465, 406, 225])
############# 51 #############
original shape of data torch.Size([512, 512, 861])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 331 liver ZMax 559
xpad 3.0 ypad 3.0 zpad 22.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 309.0 to 581.0
cropped_mask torch.Size([465, 406, 272])
cropped_volume torch.Size([465, 406, 272])
############# 52 #############
original shape of data torch.Size([512, 512, 129])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 27 liver ZMax 124
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 24.0 to 127.0
cropped_mask torch.Size([465, 406, 103])
cropped_volume torch.Size([465, 406, 103])
############# 53 #############
original shape of data torch.Size([512, 512, 172])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 44 liver ZMax 157
xpad 3.0 ypad 3.0 zpad 4.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 40.0 to 161.0
cropped_mask torch.Size([465, 406, 121])
cropped_volume torch.Size([465, 406, 121])
############# 54 #############
original shape of data torch.Size([512, 512, 534])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 303 liver ZMax 471
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 290.0 to 484.0
cropped_mask torch.Size([465, 406, 194])
cropped_volume torch.Size([465, 406, 194])
############# 55 #############
original shape of data torch.Size([512, 512, 200])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 60 liver ZMax 182
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 55.0 to 187.0
cropped_mask torch.Size([465, 406, 132])
cropped_volume torch.Size([465, 406, 132])
############# 56 #############
original shape of data torch.Size([512, 512, 91])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 19 liver ZMax 85
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 17.0 to 87.0
cropped_mask torch.Size([465, 406, 70])
cropped_volume torch.Size([465, 406, 70])
############# 57 #############
original shape of data torch.Size([512, 512, 139])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 19 liver ZMax 137
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 16.0 to 138
cropped_mask torch.Size([465, 406, 122])
cropped_volume torch.Size([465, 406, 122])
############# 58 #############
original shape of data torch.Size([512, 512, 135])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 37 liver ZMax 132
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 34.0 to 134
cropped_mask torch.Size([465, 406, 100])
cropped_volume torch.Size([465, 406, 100])
############# 59 #############
original shape of data torch.Size([512, 512, 151])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 48 liver ZMax 145
xpad 3.0 ypad 3.0 zpad 4.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 44.0 to 149.0
cropped_mask torch.Size([465, 406, 105])
cropped_volume torch.Size([465, 406, 105])
############# 60 #############
original shape of data torch.Size([512, 512, 124])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 6 liver ZMax 121
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 3.0 to 123
cropped_mask torch.Size([465, 406, 120])
cropped_volume torch.Size([465, 406, 120])
############# 61 #############
original shape of data torch.Size([512, 512, 111])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 11 liver ZMax 99
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 8.0 to 102.0
cropped_mask torch.Size([465, 406, 94])
cropped_volume torch.Size([465, 406, 94])
############# 62 #############
original shape of data torch.Size([512, 512, 122])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 22 liver ZMax 120
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 19.0 to 121
cropped_mask torch.Size([465, 406, 102])
cropped_volume torch.Size([465, 406, 102])
############# 63 #############
original shape of data torch.Size([512, 512, 132])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 33 liver ZMax 128
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 30.0 to 131.0
cropped_mask torch.Size([465, 406, 101])
cropped_volume torch.Size([465, 406, 101])
############# 64 #############
original shape of data torch.Size([512, 512, 260])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 9 liver ZMax 247
xpad 3.0 ypad 3.0 zpad 7.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 2.0 to 254.0
cropped_mask torch.Size([465, 406, 252])
cropped_volume torch.Size([465, 406, 252])
############# 65 #############
original shape of data torch.Size([512, 512, 841])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 347 liver ZMax 596
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 326.0 to 617.0
cropped_mask torch.Size([465, 406, 291])
cropped_volume torch.Size([465, 406, 291])
############# 66 #############
original shape of data torch.Size([512, 512, 122])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 26 liver ZMax 115
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 23.0 to 118.0
cropped_mask torch.Size([465, 406, 95])
cropped_volume torch.Size([465, 406, 95])
############# 67 #############
original shape of data torch.Size([512, 512, 113])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 4 liver ZMax 107
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 1.0 to 110.0
cropped_mask torch.Size([465, 406, 109])
cropped_volume torch.Size([465, 406, 109])
############# 68 #############
original shape of data torch.Size([512, 512, 125])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 6 liver ZMax 120
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 3.0 to 123.0
cropped_mask torch.Size([465, 406, 120])
cropped_volume torch.Size([465, 406, 120])
############# 69 #############
original shape of data torch.Size([512, 512, 155])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 39 liver ZMax 151
xpad 3.0 ypad 3.0 zpad 4.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 35.0 to 154
cropped_mask torch.Size([465, 406, 119])
cropped_volume torch.Size([465, 406, 119])
############# 70 #############
original shape of data torch.Size([512, 512, 119])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 1 liver ZMax 113
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 0 to 118
cropped_mask torch.Size([465, 406, 118])
cropped_volume torch.Size([465, 406, 118])
############# 71 #############
original shape of data torch.Size([512, 512, 74])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 12 liver ZMax 70
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 10.0 to 72.0
cropped_mask torch.Size([465, 406, 62])
cropped_volume torch.Size([465, 406, 62])
############# 72 #############
original shape of data torch.Size([512, 512, 124])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 28 liver ZMax 68
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 25.0 to 71.0
cropped_mask torch.Size([465, 406, 46])
cropped_volume torch.Size([465, 406, 46])
############# 73 #############
original shape of data torch.Size([512, 512, 225])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 126 liver ZMax 210
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 120.0 to 216.0
cropped_mask torch.Size([465, 406, 96])
cropped_volume torch.Size([465, 406, 96])
############# 74 #############
original shape of data torch.Size([512, 512, 537])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 312 liver ZMax 487
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 299.0 to 500.0
cropped_mask torch.Size([465, 406, 201])
cropped_volume torch.Size([465, 406, 201])
############# 75 #############
original shape of data torch.Size([512, 512, 105])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 55 liver ZMax 98
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 52.0 to 101.0
cropped_mask torch.Size([465, 406, 49])
cropped_volume torch.Size([465, 406, 49])
############# 76 #############
original shape of data torch.Size([512, 512, 96])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 61 liver ZMax 89
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 59.0 to 91.0
cropped_mask torch.Size([465, 406, 32])
cropped_volume torch.Size([465, 406, 32])
############# 77 #############
original shape of data torch.Size([512, 512, 192])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 94 liver ZMax 185
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 89.0 to 190.0
cropped_mask torch.Size([465, 406, 101])
cropped_volume torch.Size([465, 406, 101])
############# 78 #############
original shape of data torch.Size([512, 512, 239])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 135 liver ZMax 225
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 129.0 to 231.0
cropped_mask torch.Size([465, 406, 102])
cropped_volume torch.Size([465, 406, 102])
############# 79 #############
original shape of data torch.Size([512, 512, 366])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 204 liver ZMax 352
xpad 3.0 ypad 3.0 zpad 9.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 195.0 to 361.0
cropped_mask torch.Size([465, 406, 166])
cropped_volume torch.Size([465, 406, 166])
############# 80 #############
original shape of data torch.Size([512, 512, 212])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 114 liver ZMax 196
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 109.0 to 201.0
cropped_mask torch.Size([465, 406, 92])
cropped_volume torch.Size([465, 406, 92])
############# 81 #############
original shape of data torch.Size([512, 512, 216])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 132 liver ZMax 208
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 127.0 to 213.0
cropped_mask torch.Size([465, 406, 86])
cropped_volume torch.Size([465, 406, 86])
############# 82 #############
original shape of data torch.Size([512, 512, 518])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 305 liver ZMax 490
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 292.0 to 503.0
cropped_mask torch.Size([465, 406, 211])
cropped_volume torch.Size([465, 406, 211])
############# 83 #############
original shape of data torch.Size([512, 512, 541])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 324 liver ZMax 500
xpad 3.0 ypad 3.0 zpad 14.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 310.0 to 514.0
cropped_mask torch.Size([465, 406, 204])
cropped_volume torch.Size([465, 406, 204])
############# 84 #############
original shape of data torch.Size([512, 512, 168])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 83 liver ZMax 157
xpad 3.0 ypad 3.0 zpad 4.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 79.0 to 161.0
cropped_mask torch.Size([465, 406, 82])
cropped_volume torch.Size([465, 406, 82])
############# 85 #############
original shape of data torch.Size([512, 512, 94])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 57 liver ZMax 92
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 55.0 to 93
cropped_mask torch.Size([465, 406, 38])
cropped_volume torch.Size([465, 406, 38])
############# 86 #############
original shape of data torch.Size([512, 512, 198])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 122 liver ZMax 180
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 117.0 to 185.0
cropped_mask torch.Size([465, 406, 68])
cropped_volume torch.Size([465, 406, 68])
############# 87 #############
original shape of data torch.Size([512, 512, 147])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 77 liver ZMax 135
xpad 3.0 ypad 3.0 zpad 4.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 73.0 to 139.0
cropped_mask torch.Size([465, 406, 66])
cropped_volume torch.Size([465, 406, 66])
############# 88 #############
original shape of data torch.Size([512, 512, 541])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 332 liver ZMax 510
xpad 3.0 ypad 3.0 zpad 14.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 318.0 to 524.0
cropped_mask torch.Size([465, 406, 206])
cropped_volume torch.Size([465, 406, 206])
############# 89 #############
original shape of data torch.Size([512, 512, 217])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 127 liver ZMax 202
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 122.0 to 207.0
cropped_mask torch.Size([465, 406, 85])
cropped_volume torch.Size([465, 406, 85])
############# 90 #############
original shape of data torch.Size([512, 512, 343])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 214 liver ZMax 324
xpad 3.0 ypad 3.0 zpad 9.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 205.0 to 333.0
cropped_mask torch.Size([465, 406, 128])
cropped_volume torch.Size([465, 406, 128])
############# 91 #############
original shape of data torch.Size([512, 512, 519])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 323 liver ZMax 492
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 310.0 to 505.0
cropped_mask torch.Size([465, 406, 195])
cropped_volume torch.Size([465, 406, 195])
############# 92 #############
original shape of data torch.Size([512, 512, 871])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 386 liver ZMax 600
xpad 3.0 ypad 3.0 zpad 22.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 364.0 to 622.0
cropped_mask torch.Size([465, 406, 258])
cropped_volume torch.Size([465, 406, 258])
############# 93 #############
original shape of data torch.Size([512, 512, 733])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 393 liver ZMax 650
xpad 3.0 ypad 3.0 zpad 18.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 375.0 to 668.0
cropped_mask torch.Size([465, 406, 293])
cropped_volume torch.Size([465, 406, 293])
############# 94 #############
original shape of data torch.Size([512, 512, 630])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 335 liver ZMax 610
xpad 3.0 ypad 3.0 zpad 16.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 319.0 to 626.0
cropped_mask torch.Size([465, 406, 307])
cropped_volume torch.Size([465, 406, 307])
############# 95 #############
original shape of data torch.Size([512, 512, 647])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 343 liver ZMax 622
xpad 3.0 ypad 3.0 zpad 16.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 327.0 to 638.0
cropped_mask torch.Size([465, 406, 311])
cropped_volume torch.Size([465, 406, 311])
############# 96 #############
original shape of data torch.Size([512, 512, 896])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 422 liver ZMax 636
xpad 3.0 ypad 3.0 zpad 22.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 400.0 to 658.0
cropped_mask torch.Size([465, 406, 258])
cropped_volume torch.Size([465, 406, 258])
############# 97 #############
original shape of data torch.Size([512, 512, 811])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 361 liver ZMax 535
xpad 3.0 ypad 3.0 zpad 20.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 341.0 to 555.0
cropped_mask torch.Size([465, 406, 214])
cropped_volume torch.Size([465, 406, 214])
############# 98 #############
original shape of data torch.Size([512, 512, 766])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 367 liver ZMax 559
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 348.0 to 578.0
cropped_mask torch.Size([465, 406, 230])
cropped_volume torch.Size([465, 406, 230])
############# 99 #############
original shape of data torch.Size([512, 512, 549])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 343 liver ZMax 515
xpad 3.0 ypad 3.0 zpad 14.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 329.0 to 529.0
cropped_mask torch.Size([465, 406, 200])
cropped_volume torch.Size([465, 406, 200])
############# 100 #############
original shape of data torch.Size([512, 512, 751])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 301 liver ZMax 483
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 282.0 to 502.0
cropped_mask torch.Size([465, 406, 220])
cropped_volume torch.Size([465, 406, 220])
############# 101 #############
original shape of data torch.Size([512, 512, 751])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 332 liver ZMax 520
xpad 3.0 ypad 3.0 zpad 19.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 313.0 to 539.0
cropped_mask torch.Size([465, 406, 226])
cropped_volume torch.Size([465, 406, 226])
############# 102 #############
original shape of data torch.Size([512, 512, 836])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 355 liver ZMax 533
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 334.0 to 554.0
cropped_mask torch.Size([465, 406, 220])
cropped_volume torch.Size([465, 406, 220])
############# 103 #############
original shape of data torch.Size([512, 512, 696])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 265 liver ZMax 498
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 248.0 to 515.0
cropped_mask torch.Size([465, 406, 267])
cropped_volume torch.Size([465, 406, 267])
############# 104 #############
original shape of data torch.Size([512, 512, 917])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 395 liver ZMax 646
xpad 3.0 ypad 3.0 zpad 23.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 372.0 to 669.0
cropped_mask torch.Size([465, 406, 297])
cropped_volume torch.Size([465, 406, 297])
############# 105 #############
original shape of data torch.Size([512, 512, 841])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 294 liver ZMax 556
xpad 3.0 ypad 3.0 zpad 21.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 273.0 to 577.0
cropped_mask torch.Size([465, 406, 304])
cropped_volume torch.Size([465, 406, 304])
############# 106 #############
original shape of data torch.Size([512, 512, 722])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 345 liver ZMax 643
xpad 3.0 ypad 3.0 zpad 18.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 327.0 to 661.0
cropped_mask torch.Size([465, 406, 334])
cropped_volume torch.Size([465, 406, 334])
############# 107 #############
original shape of data torch.Size([512, 512, 671])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 407 liver ZMax 638
xpad 3.0 ypad 3.0 zpad 17.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 390.0 to 655.0
cropped_mask torch.Size([465, 406, 265])
cropped_volume torch.Size([465, 406, 265])
############# 108 #############
original shape of data torch.Size([512, 512, 645])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 388 liver ZMax 628
xpad 3.0 ypad 3.0 zpad 16.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 372.0 to 644.0
cropped_mask torch.Size([465, 406, 272])
cropped_volume torch.Size([465, 406, 272])
############# 109 #############
original shape of data torch.Size([512, 512, 629])
liver XMin 29 liver XMax 488 liver YMin 49 liver YMax 449 liver ZMin 379 liver ZMax 604
xpad 3.0 ypad 3.0 zpad 16.0
x cropping range 26.0 to 491.0 y cropping range 46.0 to 452.0 z cropping range 363.0 to 620.0
cropped_mask torch.Size([465, 406, 257])
cropped_volume torch.Size([465, 406, 257])

**test set for List**
############# 60 #############
original shape of data torch.Size([512, 512, 244])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 157 liver ZMax 235
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 151.0 to 241.0
cropped_mask torch.Size([435, 326, 90])
cropped_volume torch.Size([435, 326, 90])
############# 61 #############
original shape of data torch.Size([512, 512, 193])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 125 liver ZMax 185
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 120.0 to 190.0
cropped_mask torch.Size([435, 326, 70])
cropped_volume torch.Size([435, 326, 70])
############# 62 #############
original shape of data torch.Size([512, 512, 188])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 89 liver ZMax 177
xpad 3.0 ypad 3.0 zpad 5.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 84.0 to 182.0
cropped_mask torch.Size([435, 326, 98])
cropped_volume torch.Size([435, 326, 98])
############# 63 #############
original shape of data torch.Size([512, 512, 104])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 60 liver ZMax 96
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 57.0 to 99.0
cropped_mask torch.Size([435, 326, 42])
cropped_volume torch.Size([435, 326, 42])
############# 64 #############
original shape of data torch.Size([512, 512, 230])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 114 liver ZMax 217
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 108.0 to 223.0
cropped_mask torch.Size([435, 326, 115])
cropped_volume torch.Size([435, 326, 115])
############# 65 #############
original shape of data torch.Size([512, 512, 513])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 281 liver ZMax 481
xpad 3.0 ypad 3.0 zpad 13.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 268.0 to 494.0
cropped_mask torch.Size([435, 326, 226])
cropped_volume torch.Size([435, 326, 226])
############# 66 #############
original shape of data torch.Size([512, 512, 86])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 48 liver ZMax 83
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 46.0 to 85.0
cropped_mask torch.Size([435, 326, 39])
cropped_volume torch.Size([435, 326, 39])
############# 67 #############
original shape of data torch.Size([512, 512, 165])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 82 liver ZMax 160
xpad 3.0 ypad 3.0 zpad 4.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 78.0 to 164.0
cropped_mask torch.Size([435, 326, 86])
cropped_volume torch.Size([435, 326, 86])
############# 68 #############
original shape of data torch.Size([512, 512, 266])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 91 liver ZMax 154
xpad 3.0 ypad 3.0 zpad 7.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 84.0 to 161.0
cropped_mask torch.Size([435, 326, 77])
cropped_volume torch.Size([435, 326, 77])
############# 69 #############
original shape of data torch.Size([512, 512, 245])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 134 liver ZMax 229
xpad 3.0 ypad 3.0 zpad 6.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 128.0 to 235.0
cropped_mask torch.Size([435, 326, 107])
cropped_volume torch.Size([435, 326, 107])
############# 70 #############
original shape of data torch.Size([512, 512, 333])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 209 liver ZMax 318
xpad 3.0 ypad 3.0 zpad 8.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 201.0 to 326.0
cropped_mask torch.Size([435, 326, 125])
cropped_volume torch.Size([435, 326, 125])
############# 71 #############
original shape of data torch.Size([512, 512, 93])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 28 liver ZMax 85
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 26.0 to 87.0
cropped_mask torch.Size([435, 326, 61])
cropped_volume torch.Size([435, 326, 61])
############# 72 #############
original shape of data torch.Size([512, 512, 121])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 61 liver ZMax 110
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 58.0 to 113.0
cropped_mask torch.Size([435, 326, 55])
cropped_volume torch.Size([435, 326, 55])
############# 73 #############
original shape of data torch.Size([512, 512, 107])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 68 liver ZMax 99
xpad 3.0 ypad 3.0 zpad 3.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 65.0 to 102.0
cropped_mask torch.Size([435, 326, 37])
cropped_volume torch.Size([435, 326, 37])
############# 74 #############
original shape of data torch.Size([512, 512, 89])
liver XMin 39 liver XMax 468 liver YMin 89 liver YMax 409 liver ZMin 46 liver ZMax 82
xpad 3.0 ypad 3.0 zpad 2.0
x cropping range 36.0 to 471.0 y cropping range 86.0 to 412.0 z cropping range 44.0 to 84.0
cropped_mask torch.Size([435, 326, 40])
cropped_volume torch.Size([435, 326, 40])